# TBC Prediction using CNN

In [1]:
import cv2 as cv
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import os
import pickle

In [2]:
normaldir = "E:\\Download\\TB_Chest_Radiography_Database\\Normal"
tbdir = "E:\\Download\\TB_Chest_Radiography_Database\\Tuberculosis"

images = []
labels = []
imagesize = 256

In [3]:
for x in os.listdir(normaldir):
    imagedir = os.path.join(normaldir,x)
    image = cv.imread(imagedir,cv.IMREAD_GRAYSCALE)
    image = cv.resize(image,(imagesize,imagesize))
    images.append(image)
    labels.append(0)

for y in os.listdir(tbdir):
    imagedir = os.path.join(tbdir,y)
    image = cv.imread(imagedir,cv.IMREAD_GRAYSCALE)
    image = cv.resize(image,(imagesize,imagesize))
    images.append(image)
    labels.append(1)

In [4]:
images = np.array(images)
labels = np.array(labels)

imagetrain, imagetest, labeltrain, labeltest = train_test_split(images,labels,test_size=0.3,random_state=42)
imagetrain = (imagetrain.astype('float32'))/255
imagetest = (imagetest.astype('float32'))/255


In [5]:
imagetrain = imagetrain.reshape(2940, (imagesize*imagesize))

smote = SMOTE(random_state=42)
imagetrain, labeltrain = smote.fit_resample(imagetrain, labeltrain)


imagetrain = imagetrain.reshape(-1, imagesize, imagesize, 1)
print(imagetrain.shape)

(4914, 256, 256, 1)


In [6]:
print(np.unique(labeltrain,return_counts=True))

(array([0, 1]), array([2457, 2457], dtype=int64))


## CNN

In [7]:
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [8]:
cnn = keras.Sequential(
    [
    keras.Input(shape=(imagesize, imagesize, 1)),
    
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(64, activation='relu'),
    
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
    ]
)

In [9]:
cnn.compile(
    loss='binary_crossentropy',

    optimizer = keras.optimizers.Adam(learning_rate=0.001),

    metrics=['accuracy'],
)

In [10]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='accuracy',factor=0.1,patience=1,min_lr=0.00001,verbose=1)

cnn.fit(imagetrain,labeltrain,batch_size=16,epochs=10,verbose=2,callbacks=[reduce_lr])

Epoch 1/10
308/308 - 84s - loss: 0.2517 - accuracy: 0.8966 - lr: 0.0010 - 84s/epoch - 274ms/step
Epoch 2/10
308/308 - 89s - loss: 0.1103 - accuracy: 0.9611 - lr: 0.0010 - 89s/epoch - 289ms/step
Epoch 3/10
308/308 - 85s - loss: 0.0917 - accuracy: 0.9685 - lr: 0.0010 - 85s/epoch - 278ms/step
Epoch 4/10
308/308 - 81s - loss: 0.0452 - accuracy: 0.9856 - lr: 0.0010 - 81s/epoch - 263ms/step
Epoch 5/10
308/308 - 81s - loss: 0.0305 - accuracy: 0.9896 - lr: 0.0010 - 81s/epoch - 264ms/step
Epoch 6/10

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
308/308 - 81s - loss: 0.0351 - accuracy: 0.9886 - lr: 0.0010 - 81s/epoch - 264ms/step
Epoch 7/10
308/308 - 83s - loss: 0.0150 - accuracy: 0.9951 - lr: 1.0000e-04 - 83s/epoch - 268ms/step
Epoch 8/10
308/308 - 81s - loss: 0.0107 - accuracy: 0.9965 - lr: 1.0000e-04 - 81s/epoch - 263ms/step
Epoch 9/10
308/308 - 82s - loss: 0.0080 - accuracy: 0.9972 - lr: 1.0000e-04 - 82s/epoch - 266ms/step
Epoch 10/10
308/308 - 79s - loss: 0.0

In [11]:
print("TESTING DATA")
cnn.evaluate(imagetest,labeltest,batch_size=32,verbose=2)

print("ADVANCED TESTING METRICS : ")
from sklearn.metrics import classification_report, confusion_matrix
predictions = cnn.predict(imagetest,batch_size=32)
predicted_labels = (predictions > 0.5).astype('int32')
print(classification_report(labeltest,predicted_labels))
print(confusion_matrix(labeltest,predicted_labels))

TESTING DATA
40/40 - 6s - loss: 0.1256 - accuracy: 0.9738 - 6s/epoch - 143ms/step
ADVANCED TESTING METRICS : 
40/40 [==============================] - 5s 131ms/step
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1043
           1       0.92      0.93      0.92       217

    accuracy                           0.97      1260
   macro avg       0.95      0.96      0.95      1260
weighted avg       0.97      0.97      0.97      1260

[[1025   18]
 [  15  202]]


In [12]:
#with open('hasil_model_cnn.pkl','wb') as file:
    #pickle.dump(cnn,file)
#print('model berhasil disimpan ke file ','hasil_model_cnn.pkl')

In [13]:
cnn.save('hasil_model_cnn_(2.13.1).h5')  

Model berhasil disimpan ke file "hasil_model_cnn.h5"


c:\Users\MSI_PC\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
